# Option A - Generate Spheres from a CSV

In [1]:
out_dir = '/Users/cu135/hires_backdrops/cop'
mask_path = '/Users/cu135/hires_backdrops/mni/MNI152_T1_2mm_brain_mask.nii'

In [2]:
# Specify the path to your CSV file containing NIFTI paths
input_csv_path = '/Volumes/HowExp/resources/atlases/atlases/circuit_of_papez/circuit_of_papez.csv'
sheet = None # 'Sheet1'

In [3]:
from calvin_utils.permutation_analysis_utils.statsmodels_palm import CalvinStatsmodelsPalm
# Instantiate the PalmPrepararation class
cal_palm = CalvinStatsmodelsPalm(input_csv_path=input_csv_path, output_dir=out_dir, sheet=sheet)
# Call the process_nifti_paths method
data_df = cal_palm.read_and_display_data()
data_df

,sub,list_of_coordinates
0,Papez,"[0,16,7], [-21,-31,-8], [21,-31,-8], [0, -7, -..."
1,Papez Bead,"[-21, -18, -22], [-21, -21.5, -20], [-22, -25,..."


In [4]:
data_df

,sub,list_of_coordinates
0,Papez,"[0,16,7], [-21,-31,-8], [21,-31,-8], [0, -7, -..."
1,Papez Bead,"[-21, -18, -22], [-21, -21.5, -20], [-22, -25,..."



# Preprocess Your Data

**Handle NANs**
- Set drop_nans=True is you would like to remove NaNs from data
- Provide a column name or a list of column names to remove NaNs from

In [ ]:
data_df.columns

In [ ]:
coordinate_col = ['Stim_Coordinate', 'Memory_Cohen_s_D']

In [ ]:
data_df = cal_palm.drop_nans_from_columns(columns_to_drop_from=coordinate_col)
data_df

**Drop Row Based on Value of Column**

Define the column, condition, and value for dropping rows
- column = 'your_column_name'
- condition = 'above'  # Options: 'equal', 'above', 'below'

In [ ]:
column = 'Included'
condition = 'not'
value = 1

In [ ]:
data_df, other_df = cal_palm.drop_rows_based_on_value(column, condition, value)
data_df

Choose a radius (mm)

In [6]:
radius = 3

In [7]:
data_df.columns

Index(['sub', 'list_of_coordinates'], dtype='object')

In [8]:
# import pandas as pd
# data_df = pd.concat([data_df.head(14), data_df.tail(14)], ignore_index=True)

In [ ]:
import string

for i, row in data_df.iterrows():
    subname = row['Study'].translate(str.maketrans(string.punctuation, '_' * len(string.punctuation)))
    subname = subname.replace(' ', '_')
    data_df.loc[i, 'Subject'] = subname
data_df

In [ ]:
data_df['session']='memory'

A) Generate Spheres from a column that contains coordinates in lists. Ex) coord col: [x,y,z], [x,y,z]

In [9]:
from calvin_utils.nifti_utils.generate_spherical_roi import SphereROIBuilder
roi_builder = SphereROIBuilder(mask_path=mask_path, out_dir=out_dir, radius=radius)
df = roi_builder.generate_from_coordlist_column(df=data_df, coord_col='list_of_coordinates', sub_col='sub', session_col=None, project_on_brain=True, save_individual_roi=False)
df

,sub,list_of_coordinates,roi_path
0,Papez,"[0,16,7], [-21,-31,-8], [21,-31,-8], [0, -7, -...",/Users/cu135/hires_backdrops/cop/sub-Papez/ses...
1,Papez Bead,"[-21, -18, -22], [-21, -21.5, -20], [-22, -25,...",/Users/cu135/hires_backdrops/cop/sub-Papez_Bea...


B) Generate Spheres from several columns that have X, Y, and Z in them. Ex) x_col: x | y_col: y | z_col: z

In [ ]:
out_dir

In [ ]:
radius = 3

In [ ]:
from calvin_utils.nifti_utils.generate_spherical_roi import SphereROIBuilder
roi_builder = SphereROIBuilder(mask_path=None, out_dir=out_dir, radius=radius)
df = roi_builder.generate_from_separate_coord_cols(data_df, x_col='x_mni', y_col='y_mni', z_col='z_mni', sub_col='SUBJECT', session_col=None, project_on_brain=True)
df

In [ ]:
df

Save new DF with file paths

In [ ]:
df.to_excel('/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/05c_Howard_MetaAlzheimerReview_DBS-TMS_Coordinates/metadata/master_list2.xlsx', index=False)

# Option B - Generate Sphere Manually

In [ ]:
out_dir = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/rimlf/sub-teke/atrophy'
mask_path = '/Users/cu135/hires_backdrops/MNI/MNI152_T1_2mm_brain_mask.nii'

Enter coordinates of sphere

In [ ]:
x = 1.8
y = -14.3
z = -16.7
radius=5

Generate the Sphere at The Coordinate

In [ ]:
import nibabel as nib
import numpy as np
brain_img = nib.load(mask_path)
mni_arr = np.array([x,y,z])

In [ ]:
from calvin_utils.nifti_utils.generate_spherical_roi import SphereROIBuilder

SROI = SphereROIBuilder(mask_path=mask_path, out_dir=out_dir, radius=radius)
img = SROI.generate_from_single_coord(centre_xyz=[x,y,z],subject='teke', session=None, project_on_brain=True, suffix='')
img

Done
- Calvin